In [9]:
import xml.etree.ElementTree as ET
import openpyxl
from openpyxl.styles import Font
from google.colab import files, drive
import time

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Function to upload XML files to Colab
def upload_xml_files():
    uploaded = files.upload()
    file_paths = []
    for file_name, file_content in uploaded.items():
        file_path = f"/content/{file_name}"
        with open(file_path, "wb") as f:
            f.write(file_content)
        file_paths.append(file_path)
    return file_paths

# Function to extract data from XML file
def extract_data_from_xml(file_path):
    try:
        # Parse XML with namespace handling
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Define namespace for queries
        ns = {'ns': 'http://www.gsis.gr/psp/2.3'}

        # List to store extracted data
        data = []
        unique_tins = set()
        gross_total = 0
        deduction_total = 0
        employer_tax_total = 0
        netAmount1_total = 0

        # Extract totals, excluding netAmount1
        totals = root.find('.//ns:totals', ns)
        if totals is not None:
            gross_total = float(totals.attrib.get('gross', 0))
            deduction_total = float(totals.attrib.get('deduction', 0))
            employer_tax_total = float(totals.attrib.get('employerTax', 0))
            # netAmount1_total = float(totals.attrib.get('netAmount1', 0))  # Commented out to avoid double counting

        # Iterate through each <employee> element
        for employee in root.findall('.//ns:employee', ns):
            row = {}
            identification = employee.find('ns:identification', ns)
            if identification is not None:
                tin = identification.find('ns:tin', ns).text.strip() if identification.find('ns:tin', ns) is not None else ''
                unique_tins.add(tin)
                row['amm'] = identification.find('ns:amm', ns).text.strip() if identification.find('ns:amm', ns) is not None else ''

            payment = employee.find('ns:payment', ns)
            if payment is not None:
                for income in payment.findall('ns:income', ns):
                    gr_elements = income.findall('ns:gr', ns)
                    de_elements = income.findall('ns:de', ns)
                    et_elements = income.findall('ns:et', ns)

                    for gr in gr_elements:
                        kae = gr.attrib.get('kae', '')
                        code = gr.attrib.get('code', '')
                        amount = float(gr.attrib.get('amount', 0))
                        if kae:
                            row[f"a_{kae}"] = row.get(f"a_{kae}", 0) + amount
                            row[f"a_{code}"] = row.get(f"a_{code}", 0) + amount
                        else:
                            row[f"k_{code}"] = row.get(f"k_{code}", 0) + amount

                    for de in de_elements:
                        code = de.attrib.get('code', '')
                        kae = de.attrib.get('kae', '')
                        amount = float(de.attrib.get('amount', 0))
                        if code in ['4065200', '4064907']:
                            row[f"k_{code}"] = row.get(f"k_{code}", 0) + amount
                            if kae:
                                row[f"k_{kae}"] = row.get(f"k_{kae}", 0) + amount
                        elif kae:
                            row[f"a_{kae}"] = row.get(f"a_{kae}", 0) + amount
                            row[f"a_{code}"] = row.get(f"a_{code}", 0) + amount
                        else:
                            row[f"k_{code}"] = row.get(f"k_{code}", 0) + amount

                    for et in et_elements:
                        code = et.attrib.get('code', '')
                        amount = float(et.attrib.get('amount', 0))
                        row[f"et_{code}"] = row.get(f"et_{code}", 0) + amount

                netAmount1 = payment.find('ns:netAmount1', ns)
                if netAmount1 is not None:
                    row['netAmount1'] = float(netAmount1.attrib.get('value', 0))
                    netAmount1_total += row['netAmount1']

            # Append row to data list
            data.append(row)

        return data, gross_total, deduction_total, employer_tax_total, netAmount1_total, len(unique_tins)

    except Exception as e:
        print(f"Error parsing XML file {file_path}: {e}")
        return [], 0, 0, 0, 0, 0

# Function to write data to a worksheet
def write_data_to_sheet(sheet, data):
    if data:
        headers = set()
        for row_data in data:
            headers.update(row_data.keys())

        headers = sorted(headers)

        for col_num, column_title in enumerate(headers, 1):
            cell = sheet.cell(row=1, column=col_num)
            cell.value = column_title
            cell.font = Font(bold=True)

        for row_num, row_data in enumerate(data, 2):
            for col_num, value in enumerate(headers, 1):
                sheet.cell(row=row_num, column=col_num).value = row_data.get(value, "")

    else:
        sheet.cell(row=1, column=1).value = "No data available"

# Function to calculate sums for each kae or code including netAmount1
def calculate_sums(data):
    sums = {}
    netAmount1_sum = 0
    for row_data in data:
        for key, value in row_data.items():
            if key.startswith("a_") or key.startswith("k_") or key.startswith("et_"):
                if key not in sums:
                    sums[key] = 0
                sums[key] += value
        if 'netAmount1' in row_data:
            netAmount1_sum += row_data['netAmount1']
    sums['netAmount1'] = netAmount1_sum
    return sums

# Function to write sums and summary to a worksheet
def write_sums_to_sheet(sheet, sums, gross_total, deduction_total, employer_tax_total, netAmount1_total, unique_employees):
    row_num = 1
    summary_data = {
        "Gross Total": gross_total,
        "Deduction Total": deduction_total,
        "Employer Tax Total": employer_tax_total,
        "Net Amount1 Total": netAmount1_total,
        "Unique Employees": unique_employees
    }

    for key, value in summary_data.items():
        sheet.cell(row=row_num, column=1).value = key
        sheet.cell(row=row_num, column=2).value = value
        row_num += 1

    if sums:
        row_num += 1  # Leave a blank row between summary and sums
        for key, amount in sums.items():
            sheet.cell(row=row_num, column=1, value=key)
            sheet.cell(row=row_num, column=2, value=amount)
            row_num += 1
    else:
        sheet.cell(row=row_num, column=1).value = "No sums available"

# Function to aggregate and write the final summary
def write_final_summary(sheet, all_sums, all_gross, all_deduction, all_employer_tax, all_netAmount1):
    row_num = 1
    summary_data = {
        "Total Gross": sum(all_gross),
        "Total Deduction": sum(all_deduction),
        "Total Employer Tax": sum(all_employer_tax),
        "Total Net Amount1": sum(all_netAmount1)
    }

    for key, value in summary_data.items():
        sheet.cell(row=row_num, column=1).value = key
        sheet.cell(row=row_num, column=2).value = value
        row_num += 1

    if all_sums:
        row_num += 1  # Leave a blank row between summary and sums
        headers_a = sorted([key for key in all_sums if key.startswith("a_")])
        headers_k = sorted([key for key in all_sums if key.startswith("k_")])
        headers_et = sorted([key for key in all_sums if key.startswith("et_")])

        for key in headers_a + headers_k + headers_et:
            sheet.cell(row=row_num, column=1, value=key)
            sheet.cell(row=row_num, column=2, value=all_sums[key])
            row_num += 1
    else:
        sheet.cell(row=row_num, column=1).value = "No sums available"

# Check if there are XML files uploaded
print("Please upload XML files:")
xml_files = upload_xml_files()

# Output XLSX file path in Google Drive
output_path = '/content/drive/MyDrive/OLKES/files/xml_sums_str.xlsx'

# Start timing
start_time = time.time()

# Check if there are at least one XML file uploaded
if xml_files:
    # Create a new workbook
    workbook = openpyxl.Workbook()

    # Remove the default sheet created by openpyxl
    default_sheet = workbook.active
    workbook.remove(default_sheet)

    # Initialize lists to store aggregate data
    all_sums = {}
    all_gross = []
    all_deduction = []
    all_employer_tax = []
    all_netAmount1 = []

    # Process each XML file
    for idx, xml_file in enumerate(xml_files):
        # Extract data from the XML file
        data, gross_total, deduction_total, employer_tax_total, netAmount1_total, unique_employees = extract_data_from_xml(xml_file)

        # Create a new worksheet for data
        data_sheet = workbook.create_sheet(f"Data_{idx + 1}")
        write_data_to_sheet(data_sheet, data)

        # Calculate sums
        sums = calculate_sums(data)

        # Write sums and summary to a new worksheet
        sums_sheet = workbook.create_sheet(f"Sums_{idx + 1}")
        write_sums_to_sheet(sums_sheet, sums, gross_total, deduction_total, employer_tax_total, netAmount1_total, unique_employees)

        # Aggregate data for final summary
        for key, value in sums.items():
            if key not in all_sums:
                all_sums[key] = 0
            all_sums[key] += value
        all_gross.append(gross_total)
        all_deduction.append(deduction_total)
        all_employer_tax.append(employer_tax_total)
        all_netAmount1.append(netAmount1_total)

    # Create the final summary sheet
    summary_sheet = workbook.create_sheet("Summary")
    write_final_summary(summary_sheet, all_sums, all_gross, all_deduction, all_employer_tax, all_netAmount1)

    # Save the workbook
    workbook.save(output_path)
    print(f"Workbook saved to {output_path}")
else:
    print("No XML files uploaded. Please upload at least one XML file.")

# End timing
end_time = time.time()
print(f"Time taken: {end_time - start_time:.2f} seconds")


Mounted at /content/drive
Please upload XML files:


Saving XML_STR1202408.xml to XML_STR1202408 (7).xml
Saving XML_STR2202408.XML to XML_STR2202408 (7).XML
Workbook saved to /content/drive/MyDrive/OLKES/files/xml_sums_str.xlsx
Time taken: 75.15 seconds
